In [1]:
import pickle

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from tensorflow import keras
import kerasncp as kncp

import os
from typing import Iterable, Dict
import tensorflow as tf
import kerasncp as kncp
from kerasncp.tf import LTCCell, WiredCfcCell
from tensorflow import keras
import numpy as np
from matplotlib.image import imread
from tqdm import tqdm
from PIL import Image
import pandas as pd
import time
from keras_models import generate_ncp_model


2024-11-17 16:11:08.632628: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
training_root = "../../fly_to_target_dataset/diff_dataset"
val_root = "../../fly_to_target_dataset/test_data"
DROPOUT = 0.1

DEFAULT_NCP_SEED = 22222

IMAGE_SHAPE = (144, 256, 3)
IMAGE_SHAPE_CV = (IMAGE_SHAPE[1], IMAGE_SHAPE[0])

# goal_image_pth = "goal_image.png"
# goal_image = imread(goal_image_pth)
# goal_image = np.array(goal_image)
# goal_image = goal_image / 255.0

In [3]:
!export TF_CPP_MIN_LOG_LEVEL=2

In [ ]:
# batch_size = None
# seq_len = 64
# augmentation_params = None
# single_step = False
# no_norm_layer = False

# decay_rate: float = 0.85
# lr: float = 0.001
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_steps=500,
#                                                             decay_rate=decay_rate, staircase=True)
# #Adam optimizer
# optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

# gpus = tf.config.list_logical_devices('GPU')
# strategy = tf.distribute.MirroredStrategy(gpus)
# with strategy.scope():
#     mymodel = generate_ncp_model(seq_len, IMAGE_SHAPE, augmentation_params, batch_size, DEFAULT_NCP_SEED, single_step, no_norm_layer)
#     mymodel.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
#     mymodel.load_weights('../saved_models/fine_tuned_woscheduler_seed22222_lr0.0001_trainloss0.00012_valloss0.08719_diff_dataset.h5')

#     mymodel.summary()

In [4]:
def get_output_normalization(root):
    training_output_mean_fn = os.path.join(root, 'stats', 'training_output_means.csv')
    if os.path.exists(training_output_mean_fn):
        print('Loading training data output means from: %s' % training_output_mean_fn)
        output_means = np.genfromtxt(training_output_mean_fn, delimiter=',')
    else:
        output_means = np.zeros(4)

    training_output_std_fn = os.path.join(root, 'stats', 'training_output_stds.csv')
    if os.path.exists(training_output_std_fn):
        print('Loading training data output std from: %s' % training_output_std_fn)
        output_stds = np.genfromtxt(training_output_std_fn, delimiter=',')
    else:
        output_stds = np.ones(4)

    return output_means, output_stds


def load_dataset_multi(root, image_size, seq_len, shift, stride, label_scale):
    file_ending = 'png'
    IMAGE_SHAPE = (144, 256, 3)
    IMAGE_SHAPE_CV = (IMAGE_SHAPE[1], IMAGE_SHAPE[0])

    def sub_to_batch(sub_feature, sub_label):
        sfb = sub_feature.batch(seq_len, drop_remainder=True)
        slb = sub_label.batch(seq_len, drop_remainder=True)
        return tf.data.Dataset.zip((sfb, slb))
        # return sub.batch(seq_len, drop_remainder=True)
    
    # def apply_random_augmentations(image):
    #     # Generate a random number and apply augmentations with a 50% probability
    #     if tf.random.uniform(()) > 0.3:  # 30% chance to apply augmentations
    #         image = tf.image.convert_image_dtype(image, tf.float32)  # Convert to float32 for augmentation
    #         image = tf.image.random_brightness(image, max_delta=0.1)  # Random brightness adjustment
    #         image = tf.image.random_contrast(image, lower=0.8, upper=1.2)  # Random contrast adjustment
    #         image = tf.image.random_saturation(image, lower=0.8, upper=1.2)  # Random saturation adjustment
    #         image = tf.image.convert_image_dtype(image, tf.uint8)  # Convert back to uint8
    #     return image

    
    datasets = []

    #output_means, output_stds = get_output_normalization(root)

    
    for i in range(len(os.listdir(root))):
        directory = i + 1
        csv_file_name = f"{root}/{str(directory)}/data_out.csv"
        labels = np.genfromtxt(csv_file_name, delimiter=',', skip_header=1, dtype=np.float32)
        print("labels", labels)
        # if labels.shape[1] == 4:
        #     labels = (labels - output_means) / output_stds
        #     # labels = labels * label_scale
        # elif labels.shape[1] == 5:
        #     labels = (labels[:, 1:] - output_means) / output_stds
        #     # labels = labels[:,1:] * label_scale
        # else:
        #     raise Exception('Wrong size of input data (expected 4, got %d' % labels.shape[1])
    
        labels_dataset = tf.data.Dataset.from_tensor_slices(labels)
        # n_images = len(os.listdir(os.path.join(root, d))) - 1
        n_images = len([fn for fn in os.listdir(f"./{root}/{str(directory)}") if file_ending in fn])
        print(n_images)
        print("no of imgs", n_images)
        # dataset_np = np.empty((n_images, 256, 256, 3), dtype=np.uint8)
        dataset_np = np.empty((n_images, *image_size), dtype=np.uint8)

        for ix in range(n_images):
            # dataset_np[ix] = imread(os.path.join(root, d, '%06d.jpeg' % ix))
            img_file_name = root + "/" + str(directory) +'/Image' + str(ix + 1) + '.'+ file_ending
            img = Image.open(img_file_name)
            img = img.resize(IMAGE_SHAPE_CV)
            # dataset_np[ix] = img[img.height - image_size[0]:, :, :]
            dataset_np[ix] = img

        images_dataset = tf.data.Dataset.from_tensor_slices(dataset_np)
        # images_dataset = images_dataset.map(apply_random_augmentations, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = tf.data.Dataset.zip((images_dataset, labels_dataset))
        dataset = dataset.window(seq_len, shift=shift, stride=stride, drop_remainder=True).flat_map(sub_to_batch)
        datasets.append(dataset)

    return datasets

def get_dataset_multi(root, image_size, seq_len, shift, stride, validation_ratio, label_scale, extra_data_root=None):
    ds = load_dataset_multi(root, image_size, seq_len, shift, stride, label_scale)
    print('n bags: %d' % len(ds))
    cnt = 0

    for d in ds:
        for (ix, _) in enumerate(d):
            pass
            cnt += ix
    print('n windows: %d' % cnt)

    val_ix = 0

    # val_ix = int(len(ds) * validation_ratio)
    # print('\nval_ix: %d\n' % val_ix)
    # validation_datasets = ds[:val_ix]

    training_datasets = ds[val_ix:]

    # if either dataset has length 0, trying to call flat map raises error that return type is wrong
    # assert len(training_datasets) > 0 and len(validation_datasets) > 0, f"Training or validation dataset has no points!" \
    #                                                                     f"Train dataset len: {len(training_datasets)}" \
    #                                                                     f"Val dataset len: {len(validation_datasets)}"
    training = tf.data.Dataset.from_tensor_slices(training_datasets).flat_map(lambda x: x)
    # validation = tf.data.Dataset.from_tensor_slices(validation_datasets).flat_map(lambda x: x)

    # return training, validation
    return training


In [5]:
shift: int = 1
stride: int = 1
# decay_rate: float = 0.95
val_split: float = 0.2
label_scale: float = 1
seq_len = 64
val_split: float = 0.1
label_scale: float = 1

training_dataset = get_dataset_multi(training_root, IMAGE_SHAPE, seq_len, shift, stride, val_split, label_scale, extra_data_root=None)
    

labels [[-0.545209    0.5563235  -0.9627457   0.5512168 ]
 [-0.5254169   0.5640678  -0.93462795  0.53914636]
 [-0.4895641   0.5632284  -0.9060196   0.525603  ]
 ...
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]


2024-11-17 16:11:49.905598: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-11-17 16:11:49.906373: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-11-17 16:11:51.415824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-17 16:11:51.417611: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA A30 computeCapability: 8.0
coreClock: 1.44GHz coreCount: 56 deviceMemorySize: 23.49GiB deviceMemoryBandwidth: 869.04GiB/s
2024-11-17 16:11:51.417632: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-11-17 16:11:51.419327: I tensorflow/stream_executor/platform/default/dso_loa

832
no of imgs 832
labels [[-0.6370456   0.38259566 -1.0559112   0.5936858 ]
 [-0.6370456   0.38259566 -1.0559112   0.5936858 ]
 [-0.6230051   0.3925644  -1.0348189   0.5833566 ]
 ...
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
704
no of imgs 704
labels [[-0.55643934  0.24774413 -0.8744291   0.5117653 ]
 [-0.55643934  0.24774413 -0.8744291   0.5117653 ]
 [-0.5366164   0.25543112 -0.85399973  0.50109065]
 ...
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
704
no of imgs 704
labels [[-0.65295565 -0.23965634 -1.0656034   0.6283185 ]
 [-0.5059696  -0.18617387 -1.1482772   0.6241088 ]
 [-0.5059696  -0.18617387 -1.1482772   0.6241088 ]
 ...
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         

2024-11-17 16:23:03.849352: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-11-17 16:23:03.868472: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000260000 Hz


n windows: 29116864


In [6]:
print('load dataset shape', training_dataset.element_spec)
training_dataset = training_dataset.batch(64)


load dataset shape (TensorSpec(shape=(64, 144, 256, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(64, 4), dtype=tf.float32, name=None))


In [ ]:
# class PrintInputCallback(tf.keras.callbacks.Callback):
#     def on_train_batch_begin(self, batch, logs=None):
#         # Print the batch number to keep track
#         print(f"\nBatch {batch}:")

        
#         inputs, targets = next(iter(training_dataset))
#         tf.print("Input data for batch:", inputs.shape, targets.shape)

In [ ]:
# options = tf.data.Options()
# options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
# training_dataset = training_dataset.with_options(options)

# # Have GPU prefetch next training batch while first one runs
# training_dataset = training_dataset.prefetch(tf.data.AUTOTUNE)



# epochs: int = 1

# #setting validation data to None
# history = mymodel.fit(x=training_dataset, epochs=epochs,verbose=1, use_multiprocessing=False, workers=1, max_queue_size=5, callbacks=[PrintInputCallback()])
# print(history)

# # Extract the final training and validation loss
# train_loss = history.history['loss'][-1]
# # val_loss = history.history['val_loss'][-1]


# accuracy = mymodel.evaluate(x=training_dataset)
# print('Accuracy:' ,accuracy)

In [7]:
batch_size = None
seq_len = 64
augmentation_params = None
single_step = False
no_norm_layer = False

image_pth = "goal_img_diff.png"
img = Image.open(image_pth)
img = img.resize(IMAGE_SHAPE_CV)  
img_array = np.array(img) / 255
img_arrays = np.stack([img_array] * seq_len, axis = 0)
img_arrays = np.expand_dims(img_arrays, axis = 0) 
goal_image = tf.convert_to_tensor(img_arrays, dtype=tf.float32) 

In [ ]:
goal_image

In [ ]:
goal_image[0][0][0][0]

In [8]:
losses = []

In [9]:
mymodel = generate_ncp_model(seq_len, IMAGE_SHAPE, augmentation_params, batch_size, DEFAULT_NCP_SEED, single_step, no_norm_layer)
# mymodel.load_weights('saved_models/custom_loss_function.h5')

lr: float = 0.0001
decay_rate: float = 0.85
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_steps=500,
#                                                             decay_rate=decay_rate, staircase=True)

#Adam optimizer
optimizer = keras.optimizers.Adam(learning_rate=lr)

# Custom training loop
epochs = 100

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # Iterate over the training dataset
    for step, (x_batch_train, y_batch_train) in enumerate(training_dataset):
        # print(f"Step {step}, x_batch_train shape: {x_batch_train.shape}, y_batch_train shape: {y_batch_train.shape}")
        # print(x_batch_train.dtype)
        # print(x_batch_train)
        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = mymodel(x_batch_train, training=True)

            # Compute the loss between predictions and targets
            loss1 = tf.reduce_mean(tf.keras.losses.mean_squared_error(y_batch_train, y_pred))

            img_loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(tf.cast(x_batch_train, dtype=tf.float32), goal_image))
            
            total_loss = loss1 + 1e-4 * img_loss
             
            
        # Compute gradients
        gradients = tape.gradient(total_loss, mymodel.trainable_weights)
        # gradient_norms = [tf.norm(g) for g in gradients if g is not None]
        # print(f"Gradient norms: {gradient_norms}")

        # Update weights
        optimizer.apply_gradients(zip(gradients, mymodel.trainable_weights))

        print(f"Step {step}, Loss: {total_loss}")
        losses.append(total_loss)
        # break
        


Epoch 1/100


2024-11-17 16:31:37.485419: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2024-11-17 16:31:40.221690: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-11-17 16:31:40.624441: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2024-11-17 16:31:41.819810: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-11-17 16:31:44.990459: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.01GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2024-11-17 16:31:44.990521: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator 

Step 0, Loss: 0.42395126819610596
Step 1, Loss: 0.2897874712944031
Step 2, Loss: 0.25275880098342896
Step 3, Loss: 0.20641732215881348
Step 4, Loss: 0.1689286231994629
Step 5, Loss: 0.14668388664722443
Step 6, Loss: 0.1328413188457489
Step 7, Loss: 0.12358594685792923
Step 8, Loss: 0.11341056227684021
Step 9, Loss: 0.10262119770050049
Step 10, Loss: 0.0929441973567009
Step 11, Loss: 0.08455097675323486
Step 12, Loss: 0.23808524012565613
Step 13, Loss: 0.13442394137382507
Step 14, Loss: 0.10823230445384979
Step 15, Loss: 0.08514679223299026
Step 16, Loss: 0.07059036195278168
Step 17, Loss: 0.06497323513031006
Step 18, Loss: 0.06114267185330391
Step 19, Loss: 0.057849761098623276
Step 20, Loss: 0.05408960208296776
Step 21, Loss: 0.049831416457891464
Step 22, Loss: 0.15204498171806335
Step 23, Loss: 0.09989947080612183
Step 24, Loss: 0.07284393906593323
Step 25, Loss: 0.051733411848545074
Step 26, Loss: 0.04210684448480606
Step 27, Loss: 0.03924684599041939
Step 28, Loss: 0.03781939297914

In [ ]:
mymodel.save(f'saved_models/custom_loss_function_entire_dataset.h5')

In [ ]:
min(losses)

In [ ]:

plt.figure(figsize=(8, 5))
plt.plot(losses, linewidth=2)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs Epochs")
plt.grid()
plt.show()
plt.close()

In [ ]:

plt.figure(figsize=(8, 5))
# plt.plot([losses[i] for i in range(0, len(losses), 14)], linewidth=2)
# plt.plot([losses[i] for i in range(1, len(losses), 14)], linewidth=2)
# plt.plot([losses[i] for i in range(2, len(losses), 14)], linewidth=2)
plt.plot([losses[i] for i in range(3, len(losses), 14)], linewidth=2)
plt.plot([losses[i] for i in range(4, len(losses), 14)], linewidth=2)
plt.plot([losses[i] for i in range(5, len(losses), 14)], linewidth=2)
plt.plot([losses[i] for i in range(6, len(losses), 14)], linewidth=2)
# plt.plot([losses[i] for i in range(7, len(losses), 14)], linewidth=2)
# plt.plot([losses[i] for i in range(8, len(losses), 14)], linewidth=2)
# plt.plot([losses[i] for i in range(9, len(losses), 14)], linewidth=2)
# plt.plot([losses[i] for i in range(10, len(losses), 14)], linewidth=2)
# plt.plot([losses[i] for i in range(11, len(losses), 14)], linewidth=2)
# plt.plot([losses[i] for i in range(12, len(losses), 14)], linewidth=2)
# plt.plot([losses[i] for i in range(14, len(losses), 14)], linewidth=2)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs Epochs")
plt.grid()
plt.show()
plt.close()